In [34]:
import pandas as pd
import pandasql as psql
import time
import math
df_soup0=pd.read_csv("100_Match_Stats_Souporsecretv4(spell).csv")
pd.options.display.max_columns = None
prep_q1 = "SELECT * from df_soup0 where queue not in (\"5v5 ARAM games\", \"URF games\")"
df_soup = psql.sqldf(prep_q1, locals())
df_soup.head(3)

,minute,participantId,totalGold,level,xp,minionsKilled,jungleMinionsKilled,gameId,queue,teamId,win,champion_name,role,lane,summonerName,pos_x,pos_y,spell0,spell1,item0,item1,item2,item3,item4,item5
0,0,7,500,1,0,0,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,14180,14271,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer
1,1,7,500,1,0,0,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,6767,11479,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer
2,2,7,584,1,181,3,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,2170,12608,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer


In [35]:
#top_anchor =[0,14600]
#mid_anchor =[7300,7300]
#bot_anchor =[14600,0]
def lane_by_position(df,mins=[4,5,6,7,8],top_anchor = [0,14600], mid_anchor = [7300,7300], bot_anchor = [14600,0]):

    df_mins=df[df['minute'].isin(mins)].copy()
    df_mins['dist_top']=[math.dist([i,j],top_anchor) for i,j in zip(df_mins['pos_x'],df_mins['pos_y'])]
    df_mins['dist_mid']=[math.dist([i,j],mid_anchor) for i,j in zip(df_mins['pos_x'],df_mins['pos_y'])]
    df_mins['dist_bot']=[math.dist([i,j],bot_anchor) for i,j in zip(df_mins['pos_x'],df_mins['pos_y'])]
    df_mins['min_dist']=df_mins[['dist_top','dist_mid','dist_bot']].min(axis=1)

    lane_assignment=[]
    #iterate through each row
    for i in range(len(df_mins)):

        # for each row, check which distance is = to the minimum distances
        if df_mins.iloc[i].dist_bot==df_mins.iloc[i].min_dist:
            lane_assignment.append("BOTTOM")
            
        elif df_mins.iloc[i].dist_mid==df_mins.iloc[i].min_dist:
            lane_assignment.append("MIDDLE")
            
        elif df_mins.iloc[i].dist_top==df_mins.iloc[i].min_dist:
            lane_assignment.append("TOP")

    #lane_assignment
    df_mins['pos_lane']=lane_assignment
    df_mins.head(15)
    
    #make df smaller for group by
    df_mins2=df_mins[['champion_name','gameId','pos_lane']].copy()
    #take the mode of where the champion's calculated lane should be'
    position_flag_df=df_mins2.groupby(['champion_name','gameId']).agg(lambda x:x.value_counts().index[0])
    
    df_merge=df.merge(position_flag_df,on=['champion_name','gameId'])
    #want to return OG dataframe, but with another column of the new mapping
    return df_merge


beg=time.perf_counter()
w = lane_by_position(df_soup)
end=time.perf_counter()
runtime=end-beg
print(int(runtime/60), " minutes and ", runtime%60, "seconds. ")
w.head(5)

0  minutes and  2.760428800000227 seconds. 


,minute,participantId,totalGold,level,xp,minionsKilled,jungleMinionsKilled,gameId,queue,teamId,win,champion_name,role,lane,summonerName,pos_x,pos_y,spell0,spell1,item0,item1,item2,item3,item4,item5,pos_lane
0,0,7,500,1,0,0,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,14180,14271,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,TOP
1,1,7,500,1,0,0,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,6767,11479,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,TOP
2,2,7,584,1,181,3,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,2170,12608,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,TOP
3,3,7,839,3,661,11,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,1996,12305,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,TOP
4,4,7,1106,3,1054,17,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,7570,13600,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,TOP


In [36]:
# Classify the Support by item!
def support_by_item(df):
      #prep_q2 = "SELECT distinct champion_name, gameId, item0, item1, item2, item3, item4, item5 FROM df_soup where queue not in (\"5v5 ARAM games\", \"URF games\")"
      #prep2 = psql.sqldf(prep_q2, locals())

      supp_items=[ 'Frostfang','Shard of True Ice',  'Steel Shoulderguards',  'Runesteel Spaulders',  'Pauldrons of Whiterock',  'Relic Shield',
        "Targon's Buckler", 'Bulwark of the Mountain',  'Spectral Sickle', 'Harrowing Crescent', 'Black Mist Scythe']
      #should only need to check 1 row per champion / game, so let's filter on only 1 minute
      df2=df[df['minute'].isin([1])].copy()
      df2['all_items'] = df2.item0.map(str) + df2.item1.map(str) + df2.item2.map(str) + df2.item3.map(str) + df2.item4.map(str) + df2.item5.map(str) 

      support_flag=[]
      support=False
      for j in range(len(df2)):
          for si in supp_items:
              if si in df2.iloc[j]['all_items']:
                  support=True
          support_flag.append(support)
          support=False
      df2['support_flag']=support_flag
      
      #merge with original df, so that we just return the OG + the new flag
      df3=df.merge(df2[['champion_name','gameId','support_flag']],on=['champion_name','gameId'])
      return df3
      
beg=time.perf_counter()
ww=support_by_item(df_soup)
end=time.perf_counter()
runtime=end-beg
print(int(runtime/60), " minutes and ", runtime%60, "seconds. ")
ww.head(5)

0  minutes and  1.4406970000000001 seconds. 


,minute,participantId,totalGold,level,xp,minionsKilled,jungleMinionsKilled,gameId,queue,teamId,win,champion_name,role,lane,summonerName,pos_x,pos_y,spell0,spell1,item0,item1,item2,item3,item4,item5,support_flag
0,0,7,500,1,0,0,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,14180,14271,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False
1,1,7,500,1,0,0,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,6767,11479,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False
2,2,7,584,1,181,3,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,2170,12608,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False
3,3,7,839,3,661,11,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,1996,12305,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False
4,4,7,1106,3,1054,17,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,7570,13600,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False


In [37]:
# Classify the Jungler by Smite!
def jungler_by_smite(df):

        #prep_q3 = "SELECT distinct champion_name, gameId, spell0, spell1 FROM df_soup where queue not in (\"5v5 ARAM games\", \"URF games\")"
        #prep3 = psql.sqldf(prep_q3, locals())
        #prep3
        # "Smite"

        df['all_spells'] = df.spell0.map(str) + df.spell1.map(str)

        jungle_flag=[]
        jungle=False
        for j in range(len(df)):
            
            if "Smite" in df.iloc[j]['all_spells']:
                jungle=True
            jungle_flag.append(jungle)
            jungle=False
        df['jungle_flag']=jungle_flag
        df.drop(['all_spells'], axis=1,inplace=True)
        return df

beg=time.perf_counter()
www=jungler_by_smite(df_soup)
end=time.perf_counter()
runtime=end-beg
print(int(runtime/60), " minutes and ", runtime%60, "seconds. ")
www.head(5)

0  minutes and  3.8904918000007456 seconds. 


,minute,participantId,totalGold,level,xp,minionsKilled,jungleMinionsKilled,gameId,queue,teamId,win,champion_name,role,lane,summonerName,pos_x,pos_y,spell0,spell1,item0,item1,item2,item3,item4,item5,jungle_flag
0,0,7,500,1,0,0,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,14180,14271,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False
1,1,7,500,1,0,0,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,6767,11479,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False
2,2,7,584,1,181,3,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,2170,12608,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False
3,3,7,839,3,661,11,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,1996,12305,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False
4,4,7,1106,3,1054,17,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,7570,13600,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False


In [40]:
def new_lane_classification(df,mins=[4,5,6,7,8],top_anchor = [0,14600], mid_anchor = [7300,7300], bot_anchor = [14600,0]):
    df_pos=lane_by_position(df, mins, top_anchor, mid_anchor, bot_anchor)
    df_supp=support_by_item(df_pos)
    df_jungle=jungler_by_smite(df_supp)
    new_lane=[]
    new_role=[]
    for j in range(len(df_jungle)):
        lane=df_jungle.iloc[j]['pos_lane']
        role=df_jungle.iloc[j]['role']
        if df_jungle.iloc[j]['support_flag'] == True:
            lane = "BOTTOM"
            role = "DUO_SUPPORT"
        if df_jungle.iloc[j]['jungle_flag'] == True:
            lane = "JUNGLE"
            role="SOLO"
        if (lane == "BOTTOM" and role != "DUO_SUPPORT"):
            role = "DUO_CARRY"
        new_lane.append(lane)
        new_role.append(role)
    df_jungle['new_lane']=new_lane
    df_jungle['new_role']=new_role
    return df_jungle

beg=time.perf_counter()
wwww=new_lane_classification(df_soup)
end=time.perf_counter()
runtime=end-beg
print(int(runtime/60), " minutes and ", runtime%60, "seconds. ")
wwww.head(5)

0  minutes and  21.11388790000092 seconds. 


,minute,participantId,totalGold,level,xp,minionsKilled,jungleMinionsKilled,gameId,queue,teamId,win,champion_name,role,lane,summonerName,pos_x,pos_y,spell0,spell1,item0,item1,item2,item3,item4,item5,jungle_flag,pos_lane,support_flag,new_lane,new_role
0,0,7,500,1,0,0,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,14180,14271,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False,TOP,False,TOP,SOLO
1,1,7,500,1,0,0,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,6767,11479,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False,TOP,False,TOP,SOLO
2,2,7,584,1,181,3,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,2170,12608,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False,TOP,False,TOP,SOLO
3,3,7,839,3,661,11,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,1996,12305,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False,TOP,False,TOP,SOLO
4,4,7,1106,3,1054,17,0,3849016581,5v5 Draft Pick games,Red,0,Darius,SOLO,TOP,Boss 0f This Gym,7570,13600,Flash,Teleport,Thornmail,Stridebreaker,Plated Steelcaps,Sterak's Gage,Dead Man's Plate,Caulfield's Warhammer,False,TOP,False,TOP,SOLO


In [43]:
sv=wwww[['gameId','minute','champion_name','summonerName','totalGold','role','lane','new_role','new_lane']]
sv=sv.to_csv("New_Lane_data.csv",index=False)